In [218]:
import numpy as np
import matplotlib as plt
import PIL.Image as img
from pathlib import Path
working_dir = Path().resolve()
print(working_dir)

/home/vexrinawlw/Документы/projects/neuron_labs


In [219]:
def convert(file_name):
    '''
    Перевод png файла в массив тюплов RGBA
    (red green blue alpha)
    '''
    image = img.open(working_dir/file_name)
    return image.convert('RGBA').getdata()

In [220]:
def take_datas(letters):
    '''
    Перевод всех изображений в матрицу и сохранений всех матриц в отдельный массив
    '''
    letters_data = []
    for letter in letters:
        for i in range(4): # 4 - количество образов буквы
            data = convert(f'letters/{letter}{i+1}.png')
            letters_data.append(data)
    return letters_data

In [221]:
letters = ['q','h','i','w'] # буквы, для которых составлены образы

datas = take_datas(letters)
# print(len(datas))
def tuples_to_mtx(data:list[tuple])->list[list[int]]:
    '''
    Конвентирует массив тюплов в матрицу, 
    где ТОЛЬКО черный цвет с alpha=255 - единица,
    а любое другое значение - 0
    '''
    result = [[]]
    i, j = 0, 0
    for item in data:
        if j == 32:
            result.append([])
            i+=1
            j=0
        if item[0] == 0 and item[1] == 0 and item[2] == 0 and item[3] == 255:
            result[i].append(1)
            j+=1
        else:
            result[i].append(0)
            j+=1
    return result


In [222]:
mtxs=[]
for data in datas:
    mtxs.append(tuples_to_mtx(data))

In [223]:
def print_mtx(mtx):
    for row in mtx:
        print(row)

def mtx_to_vector(mtx):
    result = []
    for row in mtx:
        for item in row:
            result.append(item)
    return result

In [224]:
class Neuron:
    def __init__(self, vectors, response):
        self.weight = np.random.rand(4,1024)
        self.letter_q = vectors[0]
        self.letter_h = vectors[1]
        self.letter_i = vectors[2]
        self.letter_w = vectors[3]
    
    def weight_sum(self):
        self.after_train = [[],[],[],[]]
        for index in range(1024):
            self.after_train[0].append(self.weight[0][index]*self.letter_q[index])
            self.after_train[1].append(self.weight[1][index]*self.letter_h[index])
            self.after_train[2].append(self.weight[2][index]*self.letter_i[index])
            self.after_train[3].append(self.weight[3][index]*self.letter_w[index])
        
        self.result = []
        for item in self.after_train:
            self.result.append(sum(item))
        self.predict()

    def predict(self):
        self.guess = []
        for item in self.result:
            if item>= 0 and item<25:
                self.guess.append('q')
            elif item>= 25 and item<50:
                self.guess.append('h')
            elif item>= 50 and item<75:
                self.guess.append('i')
            elif item>= 75 and item<100:
                self.guess.append('w')

In [225]:
vector = []
for i in range(0,16,4):
    vector.append(mtx_to_vector(mtxs[i]))
N = Neuron(vector, [1,2,3,4])

import time

start = time.time()
N.weight_sum()
end = time.time()
print(end-start)
print(N.guess)
N.result

0.004606962203979492
['h', 'q', 'q', 'q']


[35.58353619887042, 9.800392011386784, 11.027084003642285, 13.737821163001529]

cθδεωαπs